In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
requests.packages.urllib3.disable_warnings()
import urllib.request
import ssl
import re
months = {'Января':'01', 'Февраля':'02','Марта':'03','Апреля':'04','Мая':'05','Июня':'06','Июля':'07',
          'Августа':'08','Сентября':'09','Октября':'10','Ноября':'11','Декабря':'11'}


In [3]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

baseurl = 'https://reestr.minsvyaz.ru'

In [5]:
#поиск количества страниц c отображением по 100
def find_n():
    req = requests.get('https://reestr.minsvyaz.ru/reestr/?show_count=100', verify=False).text
    res = BeautifulSoup(req, 'lxml').find('div', class_='page_nav_area').find_all('a', class_='nav_item')
    return int(res[3].text)+1

In [4]:
for i in range(1, find_n()):
    # Создание ссылок для всех страниц поиска на сайте реестра (с отображением по 100)
    url = baseurl + '/reestr/?PAGEN_1=' + str(i) + '&show_count=100'
    response = urllib.request.urlopen(url)
    print(url)
    # поиск ссылок на страницы с юридической информации о ПО
    links = re.findall('<a href=\"(/reestr/\d*/)\"', str(response.read()))
    with open('links.txt', mode='a', encoding='utf-8') as myfile:
        for line in links:
            print(baseurl + line, file=myfile)
    myfile.close

https://reestr.minsvyaz.ru/reestr/?PAGEN_1=1&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=2&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=3&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=4&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=5&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=6&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=7&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=8&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=9&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=10&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=11&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=12&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=13&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=14&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=15&show_count=100
https://reestr.minsvyaz.ru/reestr/?PAGEN_1=16&show_count=100
https://reestr.minsvyaz.ru/reestr

In [15]:
# константные значения
links = "links.txt"
# количество уникальных названий ПО
num = 220

# создание словаря для передачи его в последствии датафрейму в пандас
values = {'Название по': [], 'Вид организации':[],'Название организации': [], 'Все продукты организации':[], 'ИНН': [], 'Сайт': [], 'Альтернатив': [],  'Класс по': [], 'Дата регистрации':[], 'Рег.номер ПО':[],
         'Дата решения':[],'Решение':[],'Ссылка на приказ':[]}

values['Название по'] = ['' for element in range(num)]
values['Название организации'] = ['' for element in range(num)]
values['ИНН'] = ['' for element in range(num)]
values['Сайт'] = ['' for element in range(num)]
values['Альтернатив'] = ['' for element in range(num)]
values['Класс по'] = ['' for element in range(num)]
values['Вид организации'] = ['' for element in range(num)]
values['Все продукты организации'] = ['' for element in range(num)]
values['Дата регистрации'] = ['' for element in range(num)]
values['Рег.номер ПО'] = ['' for element in range(num)]
values['Дата решения'] = ['' for element in range(num)]
values['Решение'] = ['' for element in range(num)]
values['Ссылка на приказ'] = ['' for element in range(num)]
values['Сведения об исключительном праве'] = ['' for element in range(num)]


In [16]:
f = open(links)
i = 0
for url in f:
    print(url)
    print(str(i) + ' of ' + str(num))
    req = requests.get(url, verify=False).text
    soup = BeautifulSoup(req, 'lxml')
    # 1 название ПО
    soft = soup.find('h1', id='pagetitle').text
    values['Название по'][i] = soft
    # 2 название организации
    try:
        orgname = soup.find_all('a', title=re.compile("Все продукты"))
    except AttributeError:
        orgname = ""
        pass
    values['Название организации'][i] = '::'.join([i.text.strip() for i in orgname])
    # 3 ИНН ^(\d{10}|\d{12})$
    try:
        inn = soup.find_all('div', text=re.compile("(\d{10}\s|\d{12}\s)"))
    except AttributeError:
        inn = ''
        pass
    #инн может быть несколько, например в случае 4 правообладателей-людей, поэтому инн в списке
    values['ИНН'][i] = '::'.join([i.text.strip() for i in inn])
    # 4 альтернативное название организации
    try:
        altname = str(soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')).split('<br/>')
        altname = re.sub('''<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>''','',
                         altname[0]).strip() +'::'+ "::".join(altname[1:-1])
        altname = re.sub('</div>','', altname.strip('::')).strip()
    except AttributeError:
        altname = ""
        pass
    values['Альтернатив'][i] = altname.strip()
    print(altname.strip())
    # 5 Класс по
    try:
        softclas = soup.find('span', text=re.compile("Класс ПО:")).find_parent('div').find_all('font')
        softclass = '::'.join([x.text for x in softclas])
    except AttributeError:
        softclass = ""
        pass
    values['Класс по'][i] = softclass
    # 6 сайт производителя
    try:
        site = soup.find('span', text=re.compile("Сайт производителя:")).find_parent('div').find('a').get('href')
    except AttributeError:
        site = ""
        pass
    values['Сайт'][i] = site
    # 7 вид организации
    try:
        type_of_organization = soup.find("h5", attrs={ "style" : "font-size: 20px; margin-bottom: 12px;"}).text
    except AttributeError:
        type_of_organization = ""
        pass
    values['Вид организации'][i] = type_of_organization
    # 8 все продукты организации
    try:
        all_products = soup.find('a', title='Все продукты организации').get('href')
    except AttributeError:
        all_products = ""
        pass
    values['Все продукты организации'][i] = 'https://reestr.digital.gov.ru' + all_products
    # 9 дата регистрации
    try:
        date_of_registration = soup.find('span', text=re.compile("Дата регистрации:")).find_parent('div').text
        temp = re.sub('Дата регистрации:', '', date_of_registration).strip()
        temp2 = ' '.join([x for x in temp.split(' ') if x!='']).split(' ')
        temp3 = '.'.join([temp2[0],months.get(temp2[1]),temp2[2]])
    except AttributeError:
        temp3 = ""
        pass
    values['Дата регистрации'][i] = temp3
    # 10 регистрационный номер ПО
    try:
        reg_number = soup.find('span', text=re.compile("Рег. номер ПО:")).find_parent('div').text
        r = re.sub('Рег. номер ПО:', '', reg_number).strip()
    except AttributeError:
        r = ""
        pass
    print(reg_number)
    values['Рег.номер ПО'][i] = r
    # 11 дата решения
    try:
        date_of_decision = soup.find('span', text=re.compile("Дата решения")).find_parent('div').text
        tempp = re.sub('Дата решения уполномоченного органа:', '', date_of_decision).strip()
        tempp2 = ' '.join([x for x in tempp.split(' ') if x!='']).split(' ')
        tempp3 = '.'.join([tempp2[0],months.get(tempp2[1]),tempp2[2]])
    except AttributeError:
        tempp3 = ""
        pass
    values['Дата решения'][i] = tempp3
    # 12 решение
    try:
        decision = soup.find('span', text=re.compile("Решение уполномоченного")).find_parent('div').text
    except AttributeError:
        decision = ""
        pass
    values['Решение'][i] = re.sub('Решение уполномоченного органа:', '', decision).strip()
    # 13 ссылка на приказ минкомсвязи
    try:
        link = soup.find('span', text=re.compile("Ссылка на приказ")).find_parent('div').find('a').get('href')
    except AttributeError:
        link = ""
        pass
    values['Ссылка на приказ'][i] = link
    #сведения об исключительном праве
    try:
        svedeniya = soup.find('p', attrs = {'style':"line-height: 28px;"}).text.strip()
    except AttributeError:
        svedeniya = ''
        pass
    values['Сведения об исключительном праве'][i] = svedeniya
        
    i = i + 1
    if i>num-1:
        break


https://reestr.minsvyaz.ru/reestr/58191/

0 of 220
СУБД «Ред База Данных»

Рег. номер ПО:
                1            
https://reestr.minsvyaz.ru/reestr/58193/

1 of 220


Рег. номер ПО:
                2            
https://reestr.minsvyaz.ru/reestr/58192/

2 of 220


Рег. номер ПО:
                3            
https://reestr.minsvyaz.ru/reestr/61242/

3 of 220


Рег. номер ПО:
                5            
https://reestr.minsvyaz.ru/reestr/61245/

4 of 220
ЭПС "Система ГАРАНТ"

Рег. номер ПО:
                6            
https://reestr.minsvyaz.ru/reestr/61250/

5 of 220
Е1 Евфрат

Рег. номер ПО:
                8            
https://reestr.minsvyaz.ru/reestr/61248/

6 of 220
Альт Линукс СПТ, сертификат ФСТЭК

Рег. номер ПО:
                9            
https://reestr.minsvyaz.ru/reestr/61251/

7 of 220


Рег. номер ПО:
                10            
https://reestr.minsvyaz.ru/reestr/61252/

8 of 220


Рег. номер ПО:
                11            
https://reestr.minsvyaz.ru/reest

Dr.Web DSS::DSS::Dr.Web Desktop Security Suite (macOS) версия 12 ::Dr.Web Desktop Security Suite (для Windows) версия 11 ::Dr.Web Desktop Security Suite (для Linux) версия 11 ::Dr.Web Desktop Security Suite (Windows) версия 12

Рег. номер ПО:
                47            
https://reestr.minsvyaz.ru/reestr/61321/

44 of 220
Dr.Web ESS::ESS::Dr.Web Enterprise Security Suite версия 10 ::Dr.Web Enterprise Security Suite версия 6.0 ::Dr.Web Enterprise Suite. Антивирусная защита для рабочих станций и серверов Windows версия 5.0 ::Dr.Web Enterprise Security Suite версия 12

Рег. номер ПО:
                48            
https://reestr.minsvyaz.ru/reestr/61322/

45 of 220


Рег. номер ПО:
                49            
https://reestr.minsvyaz.ru/reestr/61323/

46 of 220


Рег. номер ПО:
                50            
https://reestr.minsvyaz.ru/reestr/61324/

47 of 220


Рег. номер ПО:
                51            
https://reestr.minsvyaz.ru/reestr/61325/

48 of 220


Рег. номер ПО:
          



Рег. номер ПО:
                108            
https://reestr.minsvyaz.ru/reestr/65278/

105 of 220
ДСПИ «CROS»

Рег. номер ПО:
                109            
https://reestr.minsvyaz.ru/reestr/65279/

106 of 220
FLEXTERA Framework

Рег. номер ПО:
                110            
https://reestr.minsvyaz.ru/reestr/65122/

107 of 220
Программа для ЭВМ "Средство защиты информации "vGate R2""

Рег. номер ПО:
                113            
https://reestr.minsvyaz.ru/reestr/65123/

108 of 220
Программа для ЭВМ "Средство защиты информации "vGate-S R2""

Рег. номер ПО:
                114            
https://reestr.minsvyaz.ru/reestr/65124/

109 of 220


Рег. номер ПО:
                115            
https://reestr.minsvyaz.ru/reestr/65125/

110 of 220


Рег. номер ПО:
                116            
https://reestr.minsvyaz.ru/reestr/65126/

111 of 220


Рег. номер ПО:
                117            
https://reestr.minsvyaz.ru/reestr/65127/

112 of 220


Рег. номер ПО:
                118   

Call-центр::Колл-центр::программное обеспечение для построения контактных центров::программное обеспечение центра телефонного обслуживания::программное обеспечение центра обработки вызовов::программное обеспечение обработки входящих вызовов (звонков)::программное обеспечение автоматизации исходящего обзвона

Рег. номер ПО:
                177            
https://reestr.minsvyaz.ru/reestr/65254/

171 of 220
InfoWatch CryptoStorage SОНО 2.0::InfoWatch CryptoStorage SDK 1.0

Рег. номер ПО:
                178            
https://reestr.minsvyaz.ru/reestr/65256/

172 of 220
InfoWatch  Traffic Monitor Standard Edition::Traffic Monitor 3.5 Standart

Рег. номер ПО:
                179            
https://reestr.minsvyaz.ru/reestr/65240/

173 of 220
Traffic Monitor 3.5

Рег. номер ПО:
                180            
https://reestr.minsvyaz.ru/reestr/65243/

174 of 220
Kaspersky Fraud Prevention Mobile SDK

Рег. номер ПО:
                181            
https://reestr.minsvyaz.ru/reestr/65245/


СПС КонсультантПлюс::ДИУ СПС КонсультантПлюс: Москва Проф ::ДИУ СПС Консультант Бюджетные организации ::ДИУ СПС Консультант Бизнес ::ДИУ СПС Консультант Бухгалтер смарт-комплект Оптимальный ::ДИУ СПС Консультант Универсал смарт-комплект Оптимальный ::ДИУ СПС Кодексы РФ ::ДИУ СС КонсультантБухгалтер: Версия Проф ::ДИУ СС КонсультантБухгалтер: Вопросы-ответы ::ДИУ СС Перспективы и риски споров в суде общей юрисдикции ::ДИУ СС КонсультантАрбитраж: Налоговые споры ::ДИУ СС Проверки и штрафы ::ДИУ СС НТА по охране труда и пожарной безопасности в офисах и ТЦ ::ДИУ ОП КонсультантПлюс: Конструктор договоров Модуль доступа ::СПС/БД КонсультантПлюс: Практика антимонопольной службы ::СПС/БД КонсультантПлюс: Эксперт-приложение ::СПС/БД КонсультантПлюс: Сводное региональное законодательство ::СПС/БД КонсультантПлюс: Московский выпуск ::СПС/БД КонсультантПлюс: Московская область ::СПС/БД КонсультантПлюс: Международное право ::СПС/БД КонсультантПлюс: Санкт-Петербург и Ленинградская область ::СПС/БД К



Рег. номер ПО:
                214            
https://reestr.minsvyaz.ru/reestr/65190/

208 of 220
ABBYY Compreno Products

Рег. номер ПО:
                215            
https://reestr.minsvyaz.ru/reestr/65192/

209 of 220


Рег. номер ПО:
                216            
https://reestr.minsvyaz.ru/reestr/65194/

210 of 220
ABBYY FormReader

Рег. номер ПО:
                217            
https://reestr.minsvyaz.ru/reestr/65195/

211 of 220


Рег. номер ПО:
                218            
https://reestr.minsvyaz.ru/reestr/65204/

212 of 220


Рег. номер ПО:
                221            
https://reestr.minsvyaz.ru/reestr/65206/

213 of 220


Рег. номер ПО:
                222            
https://reestr.minsvyaz.ru/reestr/65207/

214 of 220


Рег. номер ПО:
                223            
https://reestr.minsvyaz.ru/reestr/65209/

215 of 220


Рег. номер ПО:
                224            
https://reestr.minsvyaz.ru/reestr/65212/

216 of 220


Рег. номер ПО:
                225       

In [12]:
values

{'Название по': ['Система управления базами данных  «Ред База Данных»',
  '1С:Школа. Информатика, 11 класс',
  '1С:Школа. Русский язык, 5–6 класс. Лексикология'],
 'Вид организации': ['российская коммерческая организация',
  'российская коммерческая организация',
  'российская коммерческая организация'],
 'Название организации': ['ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "РЕД СОФТ"',
  'Общество с ограниченной ответственностью "1С-Паблишинг"',
  'Общество с ограниченной ответственностью "1С-Паблишинг"'],
 'Все продукты организации': ['https://reestr.digital.gov.ru/reestr/?filter_owner=56139',
  'https://reestr.digital.gov.ru/reestr/?filter_owner=55800',
  'https://reestr.digital.gov.ru/reestr/?filter_owner=55800'],
 'ИНН': ['9705000373', '7725192493', '7725192493'],
 'Сайт': ['http://reddatabase.ru/documentation/',
  'http://obr.1c.ru/educational/uchenikam/informatika-11-kl-2-e-izd-/',
  'http://obr.1c.ru/educational/uchenikam/russkiy-yazyk-5-6-kl-leksikologiya/'],
 'Альтернатив': ['СУ

In [17]:
df = pd.DataFrame(values)
df

,Название по,Вид организации,Название организации,Все продукты организации,ИНН,Сайт,Альтернатив,Класс по,Дата регистрации,Рег.номер ПО,Дата решения,Решение,Ссылка на приказ,Сведения об исключительном праве
0,Система управления базами данных «Ред База Да...,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕД ...",https://reestr.digital.gov.ru/reestr/?filter_o...,9705000373,http://reddatabase.ru/documentation/,СУБД «Ред База Данных»,Системы управления базами данных,29.01.2016,1,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,приобретение исключительного права
1,"1С:Школа. Информатика, 11 класс",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/informa...,,Системы управления процессами организации::Инф...,29.01.2016,2,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка
2,"1С:Школа. Русский язык, 5–6 класс. Лексикология",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/russkiy...,,Системы управления процессами организации::Инф...,29.01.2016,3,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка
3,1С:Комплексная автоматизация 8,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""1С-С...",https://reestr.digital.gov.ru/reestr/?filter_o...,7730643014,http://v8.1c.ru/ka/,,Системы управления процессами организации,20.02.2016,5,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Договор об отчуждении исключительного права на...
4,"Электронный периодический справочник ""Система ...",российская коммерческая организация,"ООО ""Научно-производственное предприятие ""ГАРА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7706131706,http://www.aero.garant.ru,"ЭПС ""Система ГАРАНТ""",Информационные системы для решения специфическ...,20.02.2016,6,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Свидетельство о государственной регистрации ба...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,1С:Свод отчетов 8,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""1С-С...",https://reestr.digital.gov.ru/reestr/?filter_o...,7730643014,http://v8.1c.ru/svod/,,Системы управления процессами организации::Инф...,18.03.2016,224,18.03.2016,Приказ Минкомсвязи России от 18.03.2016 №112,https://reestr.minsvyaz.ru/upload/iblock/1a6/1...,Договор об отчуждении исключительного права на...
216,BI-Sphere,российская коммерческая организация,"Общество с ограниченной ответственностью ""Теле...",https://reestr.digital.gov.ru/reestr/?filter_o...,7705663948,https://teleformis.ru/web/guest/bi-sphere,,Системы мониторинга и управления::Прикладное п...,18.03.2016,225,18.03.2016,Приказ Минкомсвязи России от 18.03.2016 №112,https://reestr.minsvyaz.ru/upload/iblock/1a6/1...,"Основания в результате собственной разработки,..."
217,LIMS myLab,российская коммерческая организация,"Общество с ограниченной ответственностью ""ЛИКО...",https://reestr.digital.gov.ru/reestr/?filter_o...,7204182136,https://likoris.com/lims/,,"Системы управления проектами, исследованиями, ...",18.03.2016,226,18.03.2016,Приказ Минкомсвязи России от 18.03.2016 №112,https://reestr.minsvyaz.ru/upload/iblock/1a6/1...,Свидетельство о государственной регистрации пр...
218,IQ300,российская коммерческая организация,"Общество с ограниченной ответственностью ""Айкь...",https://reestr.digital.gov.ru/reestr/?filter_o...,1650255336,http://www.iq300.ru,IQ300,Системы мониторинга и управления::Офисные прил...,18.03.2016,227,18.03.2016,При

In [18]:

#запись датафрейма в csv-файл с кодировкой utf-8
df.to_excel('Reestr.xlsx', index = False)


In [54]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = str(soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')).split('<br/>')
altname = re.sub('''<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>''','',
               altname[0]).strip() +'::'+ "::".join(altname[1:-1])
altname = re.sub('</div>','', altname.strip('::')).strip()

'1С-Битрикс24: Корпоративный портал - Холдинг::1С-Битрикс24 (Холдинг)::1С-Битрикс24::«1С-Битрикс24». Лицензия Энтерпрайз'

In [10]:
#преобразование словаря в датафрейм
values['Рег.номер ПО']

['2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2']

In [58]:
pd.DataFrame(values)

,Название по,Вид организации,Название организации,Все продукты организации,ИНН,Сайт,Альтернатив,Класс по,Дата регистрации,Рег.номер ПО,Дата решения,Решение,Ссылка на приказ,Сведения об исключительном праве
0,Система управления базами данных «Ред База Да...,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕД ...",https://reestr.digital.gov.ru/reestr/?filter_o...,9705000373,http://reddatabase.ru/documentation/,СУБД «Ред База Данных»,Системы управления базами данных,29.01.2016,1,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
1,"1С:Школа. Информатика, 11 класс",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/informa...,,Системы управления процессами организации::Инф...,29.01.2016,2,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
2,"1С:Школа. Русский язык, 5–6 класс. Лексикология",российская коммерческая организация,"Общество с ограниченной ответственностью ""1С-П...",https://reestr.digital.gov.ru/reestr/?filter_o...,7725192493,http://obr.1c.ru/educational/uchenikam/russkiy...,,Системы управления процессами организации::Инф...,29.01.2016,3,29.01.2016,Приказ Минкомсвязи России от 29.01.2016 №19,https://reestr.minsvyaz.ru/upload/iblock/88a/1...,Собственная разработка.\n \n Программный проду...
3,1С:Комплексная автоматизация 8,российская коммерческая организация,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""1С-С...",https://reestr.digital.gov.ru/reestr/?filter_o...,7730643014,http://v8.1c.ru/ka/,,Системы управления процессами организации,20.02.2016,5,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
4,"Электронный периодический справочник ""Система ...",российская коммерческая организация,"ООО ""Научно-производственное предприятие ""ГАРА...",https://reestr.digital.gov.ru/reestr/?filter_o...,7706131706,http://www.aero.garant.ru,"ЭПС ""Система ГАРАНТ""",Информационные системы для решения специфическ...,20.02.2016,6,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
5,Система электронного документооборота и автома...,российская коммерческая организация,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОГНИТИВ""",https://reestr.digital.gov.ru/reestr/?filter_o...,7728742674,http://www.evfrat.ru,Е1 Евфрат,Офисные приложения::Прикладное программное обе...,20.02.2016,8,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
6,Альт Линукс СПТ,российская коммерческая организация,"Общество с ограниченной ответственностью ""Своб...",https://reestr.digital.gov.ru/reestr/?filter_o...,5003088997,http://sptlinux.ru/,"Альт Линукс СПТ, сертификат ФСТЭК",Операционные системы,20.02.2016,9,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
7,ABBYY ScanDifFinder SDK,"российская коммерческая организация, имеющая в...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АБИ ...",https://reestr.digital.gov.ru/reestr/?filter_o...,7723594937,http://www.abbyy.ru/scandiffinder/,,Библиотеки подпрограмм (SDK)::Лингвистическое ...,20.02.2016,10,20.02.2016,Приказ Минкомсвязи России от 20.02.2016 №63,https://reestr.minsvyaz.ru/upload/iblock/2df/6...,Собственная разработка.\n \n Программный проду...
8,ABBYY PassportReader SDK,"российская коммерческая организация, имеющая в...","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АБИ ...",https://reestr.digital.gov.ru/reestr/?filter_o...,7723594937,http://www.abbyy.ru/p

In [7]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
string = re.sub('Альтернативные наименования:', '', altname.text) + ",".join(list_[0:-1])
string.strip()

'1С-Битрикс24: Корпоративный портал - Холдинг1С-Битрикс24 (Холдинг)1С-Битрикс24«1С-Битрикс24». Лицензия Энтерпрайз <div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\t1С-Битрикс24: Корпоративный портал - Холдинг,1С-Битрикс24 (Холдинг),1С-Битрикс24,«1С-Битрикс24». Лицензия Энтерпрайз'

In [17]:
req = requests.get('https://reestr.digital.gov.ru/reestr/61265/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
re.sub('<div style="margin-bottom: 16px;">\n<span style="font-size: '
       '15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\t',
       '',",".join(list_))

'1С-Битрикс24: Корпоративный портал - Холдинг,1С-Битрикс24 (Холдинг),1С-Битрикс24,«1С-Битрикс24». Лицензия Энтерпрайз, </div>'

In [15]:
req = requests.get('https://reestr.digital.gov.ru/reestr/58191/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div')
list_ = str(altname).split('<br/>')
list_

['<div style="margin-bottom: 16px;">\n<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>\n\t\t\t\t\tСУБД «Ред База Данных»\t\t\t\t</div>']

In [67]:
re.sub('Альтернативные наименования:', '', altname.text)

'\n\n\t\t\t\t\t1С-Битрикс24: Корпоративный портал - Холдинг1С-Битрикс24 (Холдинг)1С-Битрикс24«1С-Битрикс24». Лицензия Энтерпрайз '

In [29]:
#ЭТО РАБОТАЕТ!!!!!
req = requests.get('https://reestr.digital.gov.ru/reestr/58193/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
altname = soup.find('span', text=re.compile("Альтернативные наименования:")).find_parent('div').children
li = [str(x).strip() for x in altname]
li = list(filter(lambda x: x!= '<br/>', li))
re.sub('<span style="font-size: 15px; color: #777777; display: block; font-weight: bold;">Альтернативные наименования:</span>',
       '',','.join(li)).strip(',')

AttributeError: 'NoneType' object has no attribute 'find_parent'

In [41]:

req = requests.get('https://reestr.digital.gov.ru/reestr/58193/', verify=False).text
soup = BeautifulSoup(req, 'lxml')
softclas = soup.find('span', text=re.compile("Класс ПО:")).find_parent('div').find_all('font')
softclass = '::'.join([x.text for x in softclas])

'Системы управления процессами организации::Информационные системы для решения специфических отраслевых задач'

In [44]:
svedeniya = soup.find('p', attrs = {'style':"line-height: 28px;"}).text.strip()

'Собственная разработка'

In [12]:
import pandas as pd
values_ = {'a':[1,2,3,4,5],'b':[1,2,3,4,5],'c':[4,5,6,7,8]}
df_ = pd.DataFrame(values_)
df_.to_excel('values__.xlsx')